## Домашнее задание к лекции «Продвинутый pandas»

In [1]:
import pandas as pd
import re

### Задание 1
Для датафрейма log из материалов занятия создайте столбец source_type по правилам:

- если источник traffic_source равен Yandex или Google, то в source_type ставится organic;
- для источников paid и email из России ставим ad;
- для источников paid и email не из России ставим other;
- все остальные варианты берём из traffic_source без изменений.

In [2]:
data = pd.read_csv('C:\\Users\\alexander.dubovik\\Documents\\PYDA-38\\NUMP_PAND\\hw\\pandas_hw2\\visit_log.csv', sep=';')

In [45]:
## Данные для проверки итогов
data.pivot_table(index='region', columns='traffic_source', values='timestamp', aggfunc='count', margins=True)

traffic_source,direct,email,google,paid,yandex,All
region,,,,,,
Australia,222,115,91,74,88,590
Belarus,998,569,376,368,446,2757
China,397,252,151,138,171,1109
India,134,82,55,39,59,369
Russia,4052,2209,1435,1581,1636,10913
Ukraine,1210,648,399,443,500,3200
All,7013,3875,2507,2643,2900,18938


In [ ]:
def traffic(row):
    if (row['traffic_source'] == 'yandex' or row['traffic_source'] == 'google'):
        return "organic"
    elif (row['traffic_source'] == 'paid' or row['traffic_source'] == 'email'):
        if row['region'] == "Russia" :
            return "ad"
        else:
            return "other"
    else:
        return row['traffic_source']

In [47]:
data['source_type'] = data.apply(traffic, axis = 1)

In [48]:
# Проверка, ad - все кроме России NaN, other - Россия NaN, а так же можем сравнить что изменилось, 
# и итоги по странам должны быть идентичные тем, что были в первом запросе
data.pivot_table(index='region', columns='source_type', values='timestamp', aggfunc='count', margins=True)

source_type,ad,direct,organic,other,All
region,,,,,
Australia,NaN,222.0,179.0,189.0,590
Belarus,NaN,998.0,822.0,937.0,2757
China,NaN,397.0,322.0,390.0,1109
India,NaN,134.0,114.0,121.0,369
Russia,3790.0,4052.0,3071.0,NaN,10913
Ukraine,NaN,1210.0,899.0,1091.0,3200
All,3790.0,7013.0,5407.0,2728.0,18938


### Задание 2
В файле URLs.txt содержатся URL страниц новостного сайта. Вам нужно отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри URL конструкцию: /, затем 8 цифр, затем дефис. Выполните действия:

- Прочитайте содержимое файла с датафрейм.
- Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствие с заданным шаблоном.

In [86]:
df = pd.read_csv('C:\\Users\\alexander.dubovik\\Documents\\PYDA-38\\NUMP_PAND\\hw\\pandas_hw2\\URLs.txt', delimiter = "\t")
df.head()

,url
0,/world/
1,/latest/
2,/?updated=top
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...


In [108]:
df.loc[lambda x: x['url'].str.contains(r'[/]\d{8}[-]', case=False)]

,url
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...
...,...
88,/cis/35984145-kreml-prokommentiroval-soobschen...
89,/video/36071019-olimpiyskie-obekty-rio-prevrat...
90,/science/36151301-nazvano-posledstvie-zloupotr...
91,/incidents/36027330-vospitatelnitsu-zatravili-...


### Задание 3
Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значениями столбца timestamp для данного значения userId.

In [29]:
ratings = pd.read_csv('C:\\Users\\alexander.dubovik\\Documents\\PYDA-38\\NUMP_PAND\\hw\\pandas_hw2\\ratings.csv')
ratings.head(7)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187


In [30]:
rating_agg = ratings.groupby(['userId']).agg({'count', 'min', 'max'}).reset_index()
rating_agg.head()

userId movieId              rating              timestamp                  
             min count    max    min count  max         min count         max
0      1      31    20   3671    1.0    20  4.0  1260759108    20  1260759205
1      2      10    76    720    1.0    76  5.0   835355395    76   835356246
2      3      60    51  84236    2.0    51  5.0  1298861589    51  1298932787
3      4      10   204   4006    1.0   204  5.0   949778714   204   949982274
4      5       3   100  48385    1.5   100  5.0  1163373044   100  1163375145

In [31]:
def time_dif(row):
    return row['timestamp']['max'] - row['timestamp']['min']

In [32]:
rating_agg['time_dif'] = rating_agg.apply(time_dif, axis = 1)
rating_agg.head()

userId movieId              rating              timestamp                    \
             min count    max    min count  max         min count         max   
0      1      31    20   3671    1.0    20  4.0  1260759108    20  1260759205   
1      2      10    76    720    1.0    76  5.0   835355395    76   835356246   
2      3      60    51  84236    2.0    51  5.0  1298861589    51  1298932787   
3      4      10   204   4006    1.0   204  5.0   949778714   204   949982274   
4      5       3   100  48385    1.5   100  5.0  1163373044   100  1163375145   

   time_dif  
             
0      97.0  
1     851.0  
2   71198.0  
3  203560.0  
4    2101.0

In [52]:
rating = rating_agg[(rating_agg['rating']['count'] > 100)]
final = rating.mean()

In [55]:
pd.to_datetime(final['time_dif'], unit='ms')

   1970-01-01 11:08:00.507449612
dtype: datetime64[ns]

### Задание 4
Дана статистика услуг перевозок клиентов компании по типам (см. файл “Python_13_join.ipynb” в разделе «Материалы для лекции “Продвинутый pandas”» ---- Ноутбуки к лекции «Продвинутый pandas»).
Нужно сформировать две таблицы:

- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента;
- аналогичную таблицу по типам выручки с указанием адреса клиента.

Дана статистика услуг перевозок клиентов компании по типам:
- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

In [56]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [57]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [58]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [59]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [67]:
# Вариант 1 без адреса
first_table = (rzd
               .merge(auto, how='outer', on='client_id')  # пояснения
               .merge(air, how='outer', on='client_id')
              )
first_table

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [70]:
# Вариант 2 без адреса
from functools import reduce
first_table_2 = reduce(lambda a, b: a.merge(b, how='outer', on='client_id'), [rzd, auto,air])
first_table_2

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [76]:
# Вариант 1 с адресом
table_with_address = (rzd
                      .merge(auto, how='outer', on='client_id')  # пояснения
                      .merge(air, how='outer', on='client_id')
                      .merge(client_base, how='outer', on='client_id')
                     )
table_with_address

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,NaN,NaN,Комсомольская 4
1,112,2810.0,NaN,NaN,Энтузиастов 8а
2,113,10283.0,57483.0,NaN,Левобережная 1а
3,114,5774.0,83.0,NaN,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,NaN,4834.0,4.0,Строителей 18
6,117,NaN,98.0,13.0,Панфиловская 33
7,118,NaN,NaN,173.0,Мастеркова 4


In [75]:
# Вариант 2 с адресом
table_with_address_2 = reduce(lambda a, b: a.merge(b, how='outer', on='client_id'), [rzd, auto,air, client_base])
table_with_address_2

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,NaN,NaN,Комсомольская 4
1,112,2810.0,NaN,NaN,Энтузиастов 8а
2,113,10283.0,57483.0,NaN,Левобережная 1а
3,114,5774.0,83.0,NaN,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,NaN,4834.0,4.0,Строителей 18
6,117,NaN,98.0,13.0,Панфиловская 33
7,118,NaN,NaN,173.0,Мастеркова 4
